In [143]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time as t

%matplotlib inline

In [144]:
# Setting the threshold, where to stop the calculation
threshold = 0.1

### Loading the .npy files

In [145]:
masses = np.load('files/masses.npy')
positions = np.load('files/positions.npy')
velocities = np.load('files/velocities.npy')

print(positions.shape)

(100, 2)


In [146]:
#use s to increase or reduce 
s = len(positions)

position_x = positions[:,0][0:s]
position_y = positions[:,-1][0:s]
initial_velocity_x = velocities[:,0][0:s]
initial_velocity_y = velocities[:,-1][0:s]
acc_x = np.zeros(len(position_x))[0:s]
acc_y = np.zeros(len(position_y))[0:s]
mass = masses.reshape(len(masses),)[0:s]

### Defining constants and placeholders required

In [147]:
t2_cons = tf.constant(0.5,dtype=tf.float64,name='cons_fac')
time  = tf.constant(0.0001,dtype=tf.float64,name="time_step")
G = tf.constant(6.67*pow(10,5),dtype=tf.float64,name="G-constant")
neg = tf.constant(-1.0,dtype=tf.float64)

In [148]:
px = tf.placeholder(tf.float64,name="px")
py = tf.placeholder(tf.float64,name="py")
ux = tf.placeholder(tf.float64,name="ux")
uy = tf.placeholder(tf.float64,name="uy")
ax = tf.placeholder(tf.float64,name="ax")
ay = tf.placeholder(tf.float64,name="ay")
m =  tf.placeholder(tf.float64,name="mass")

#### Calculating initial euclidean distance and cube of distance for acceleration using broadcasting for outer subtraction

In [149]:
#for x
dis_x = tf.transpose([px]) - px

#for y
dis_y = tf.transpose([py]) - py

dis_z = tf.sqrt(tf.add(tf.square(dis_x),tf.square(dis_y)))

cube_dis_z = tf.pow(dis_z,3)

#### Calculating acceleration in x and y direction for each particle due to graviational force

In [150]:
#For ax
divide_mass_mul_x = tf.divide(tf.multiply(m,dis_x),cube_dis_z)

#removing nan, where cube_dis_z is zero
divide_mass_mul_x = tf.where(tf.is_nan(divide_mass_mul_x), tf.zeros_like(divide_mass_mul_x), divide_mass_mul_x)

#summation of all acceleration and multiplied with G(graiational constant)
sum_mass_mul_x = tf.multiply(G,tf.reduce_sum(divide_mass_mul_x,1))

#Final acceleration along x direction
final_acc_x = tf.multiply(neg,sum_mass_mul_x)

########################################################


#For ay
divide_mass_mul_y = tf.divide(tf.multiply(m,dis_y),cube_dis_z)

#removing nan, where cube_dis_z is zero
divide_mass_mul_y = tf.where(tf.is_nan(divide_mass_mul_y), tf.zeros_like(divide_mass_mul_y), divide_mass_mul_y);

#summation of all acceleration and multiplied with G(graiational constant)
sum_mass_mul_y = tf.multiply(G,tf.reduce_sum(divide_mass_mul_y,1))

# Final acceleration along x direction
final_acc_y = tf.multiply(neg,sum_mass_mul_y)

#### Updating position of particle along x and y direction according to eqn

\begin{align}
\ x & = \ x_0 + ut+0.5at^2  \\
\end{align}

In [151]:
## For x position
t1_x = tf.multiply(ux,time,name="t1_x")
t2_x = tf.multiply(tf.multiply(t2_cons,final_acc_x),tf.square(time))

tx = tf.add(px,tf.add(t1_x,t2_x))

In [152]:
# For y position
t1_y = tf.multiply(uy,time,name="t1_y")
t2_y = tf.multiply(tf.multiply(t2_cons,final_acc_y),tf.square(time))

ty = tf.add(py,tf.add(t1_y,t2_y))

#### Updating velocity after one time step

\begin{align}
\ v & = \ u + at  \\
\end{align}

In [153]:
# for x direction
v_x = tf.add(ux,tf.multiply(final_acc_x,time))

# for y direction
v_y = tf.add(uy,tf.multiply(final_acc_y,time))

#### Computation of distance matrix again to get minimum distance and check for threshold

In [154]:
#for x
dis_x_new = tf.transpose([tx]) - tx

#for y
dis_y_new = tf.transpose([ty]) - ty

dis_z_new = tf.sqrt(tf.add(tf.square(dis_x_new),tf.square(dis_y_new)))

max_dis_z_new = tf.reduce_max(dis_z_new)

#change the diagonal matrix to find min element and replacing with max matrix element.Because diagonal is zero
dis_z_new = tf.matrix_set_diag(dis_z_new,s*[max_dis_z_new])

min_dis_z_new = tf.reduce_min(dis_z_new)

#### Initial values in placeholders

In [161]:
feed_dict = {
    px:position_x,
    py:position_y,
    ux:initial_velocity_x,
    uy:initial_velocity_y,
    ax:acc_x,
    ay:acc_y,
    m:mass
}

In [162]:
def plot(x,y,counter):
    plt.ylim(-10, 80)
    plt.xlim(-10, 80)
    
    #plt.scatter(x, y)
    plt.scatter(x[8], y[8],c='red',s=50)
    plt.scatter(x[91], y[91],c='blue',s=50)
    plt.savefig('As1Q2_vid/'+str(counter)+'.png')
    plt.close()

In [163]:
start = t.time()

count_iter = 0

with tf.Session() as session:
    
    while(True):
        result = session.run([tx,ty,v_x,v_y,final_acc_x,final_acc_y,min_dis_z_new],feed_dict=feed_dict)
        
        # Writing graph summary in logs
        #writer = tf.summary.FileWriter("./logs/graph",session.graph)
        
        ##Updating the values
        feed_dict[px]=result[0]
        feed_dict[py]=result[1]
        feed_dict[ux]=result[2]
        feed_dict[uy]=result[3]
        feed_dict[ax]=result[4]
        feed_dict[ay]=result[5]
        
        #Save plot for the video
        #plot(result[0],result[1],count_iter)
        
        ### checking if minimum distance between any two points is below threshold
        if result[6]<=threshold:
            break
            
        count_iter+=1
        
end = t.time()
print("Time Taken: ",end - start)

Time Taken:  2.1669862270355225


#### Save position and velocity as .npy file

In [164]:
final_pos = []
final_vel = []

for i in range(len(result[0])):
    final_pos.append([result[0][i],result[1][i]])
    final_vel.append([result[2][i],result[3][i]])

In [165]:
final_pos = np.array(final_pos,dtype='float64')
final_vel = np.array(final_vel,dtype='float64')

In [166]:
np.save('output/positions.npy', final_pos)
np.save('output/velocities.npy', final_vel)

## Tensorbaord Visualization
- tensorboard --logdir=path/to/log-directory

## Video Creation

In [139]:
import os
import cv2

In [112]:
f=os.listdir("As1Q2_vid")

sorted_files = sorted(f,key=lambda x:int(x.split('.')[0]))

img_frame = []

for file in sorted_files:
    path="As1Q2_vid/"
    img = cv2.imread(path+file)
    h,w,l = img.shape
    size=(w,h)
    img_frame.append(img)
    
out = cv2.VideoWriter("As1Q2.mp4",0x7634706d,10,size)

In [137]:
for i in range(len(img_frame)):
    out.write(img_frame[i])
out.release()